# Summer Data science project


### Introduciton
The goal of this probability and statistics project is aimed towards holistic growth of my abilities to work on a data science project from start to finish in hopes of replicating the process of a corporate job to better prepare me for the industry. The difference between this project and the projects ive worked on previously will be the emphasis on project planning, data collection, optimization, documentation and producitionization things I have previously ignored. 

## Table of Contents.
1. Project planning and research. 
2. Scraping data
3. Data cleaning
4. Exploratory analysis
5. model building
6. Putting model into production (producitionization)
7. documentation. 



## 1. Project planning and research.
#### Project Ideas

    1. Weightlifting
        -Personal Record classification model (not alot of accessible data online) Can use web scraping, (openpowerlifting provides good powerlifting data.) you can add more data through research of powerlifters youtube videos(youtube api). 
        -although not big of a runner myself maybe route reccomendations through (strava api)
    2. Music 
        -Music Reccomendation model  (spotify api)
    3. Reading 
        -book reccomendation model For Ellie( Google Books API, Open Library API)
    4. Video games
        -Video game stats, Match making rank models, champion patch analysis, 
    5. any project resulting from Data from US census 
        -https://www.census.gov/data/developers/data-sets.html



 

I created a list of activities I'm passionate about. Although each subject excites me, I found myself particularly drawn to developing a music recommendation model. This project resonates with me deeply because I frequently listen to music, and I have easy access to my own listening data through the spotify api. Additionally, the availability of APIs and online data for this area makes it a practical and engaging choice for my research.

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import pandas as pd
import time
import requests
from requests.exceptions import RequestException
from spotipy.oauth2 import SpotifyClientCredentials
import time
import requests.exceptions
from spotipy.exceptions import SpotifyException


## Spotify API reserach


In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="XXXXXXX",
                                               client_secret="XXXXXXX",
                                               redirect_uri="http://localhost:8888/callback",
                                               scope="user-library-read"))
results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Dirty South  –  Find a Way (feat. Rudy)
1 Nadia Ali  –  Pressure - Alesso Radio Edit
2 Otto Knows  –  Next to Me
3 The Kid LAROI  –  BABY I'M BACK
4 Tiffany Day  –  FAR AWAY
5 Sawyer Hill  –  High On My Lows
6 Zedd  –  Lucky (feat. Remi Wolf)
7 Gryffin  –  WHERE ARE YOU TONIGHT (feat. ZOHARA)
8 Gryffin  –  LONELY (feat. Calle Lehmann)
9 Glass Animals  –  How I Learned To Love The Bomb
10 BKAYE  –  All In My Head
11 Glass Animals  –  whatthehellishappening?
12 Glass Animals  –  Show Pony
13 Childish Gambino  –  Lithonia
14 Martin Garrix  –  Carry You
15 Flux Pavilion  –  On Repeat
16 Kai Wachi  –  Happier By Now
17 INHXLE  –  ULTRA INSTINCT FUNK
18 Subtronics  –  Spacetime - Virtual Riot Remix
19 Gryffin  –  LAST OF US (feat. Rita Ora) (Jerro Remix)


This is how Spotipy reccomends we set up our API through the github. The issue with this is we need to create a new authentication for every different scope we want to call. 

In [5]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="XXXXXXX",
                                               client_secret="XXXXXXX",
                                               redirect_uri="http://localhost:8888/callback",
                                               scope="user-top-read"))

ranges = ['short_term', 'medium_term', 'long_term']

for sp_range in ranges:
    print("range:", sp_range)
    results1 = sp.current_user_top_tracks(time_range=sp_range, limit=10)
    for i, item in enumerate(results1['items']):
        print(i, item['name'], '//', item['artists'][0]['name'])
    print()


range: short_term
0 WHERE ARE YOU TONIGHT (feat. ZOHARA) // Gryffin
1 Carry You // Martin Garrix
2 All In My Head // BKAYE
3 whatthehellishappening? // Glass Animals
4 Lucky (feat. Remi Wolf) // Zedd
5 Lithonia // Childish Gambino
6 High On My Lows // Sawyer Hill
7 Happier By Now // Kai Wachi
8 Show Pony // Glass Animals
9 LONELY (feat. Calle Lehmann) // Gryffin

range: medium_term
0 Dawn // Eliminate
1 Cottage Gore // Subtronics
2 Spacetime - Virtual Riot Remix // Subtronics
3 far away - Crankdat Remix // Mark Tuan
4 Under Control (feat. Hurts) // Calvin Harris
5 Huntin’ Wabbitz // J. Cole
6 Black Out Days - Subtronics Remix // Phantogram
7 Wake Me Up // Avicii
8 how2fly // ISOxo
9 Fever // J. Cole

range: long_term
0 Delilah (pull me out of this) // Fred again..
1 Danielle (smile on my face) // Fred again..
2 ten // Fred again..
3 Clara (the night is dark) // Fred again..
4 RATATA // Skrillex
5 King Of Nothing // BoyWithUke
6 Cottage Gore // Subtronics
7 Dawn // Eliminate
8 TELEKINES

 What we need to accomplish now is to extract different features from my personal spotify data in order to determine what characteristics im drawn towards when creating my model. Additioanlly what ive noticed is that I need to create a new authorization scope each time I went to extract different types of datas instead i can 

 What we need to accomplish is to authenticate with the combined scope. This ensures that the client is authorized for all the required operations.

In [6]:
# Define all necessary scopes
scopes = [
    'user-top-read',
    'playlist-modify-private',
    'playlist-modify-public'
]

scope = ' '.join(scopes)

# Set up Spotify API client with multiple scopes
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id='XXXXXXX',
    client_secret='XXXXXXX',
    redirect_uri='http://localhost:8888/callback',
    scope=scope
))

In [7]:
# Getting features for each song
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

In [8]:
playlists = sp.user_playlists('spotify')
spotify_playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        spotify_playlist_ids.append(playlist['uri'][-22:])
    if playlists['next']:       
        playlists = sp.next(playlists)  
    else:
      playlists = None
spotify_playlist_ids[:20]

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [9]:
len(spotify_playlist_ids)

1021

In [10]:
def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item.get('track')
        if track is not None:
            ids.append(track.get('id'))





In [11]:
ids = []
for x in spotify_playlist_ids[:200]:
    getTrackIDs(x)

print(ids[:5])


['7221xIgOnuakPdLqT0F3nP', '6dOtVTDdiauQNBQEDOtlAB', '5N3hjp1WNayUPZrA8kJmJP', '0WbMK4wrZ1wFSty9F7FCgu', '3WOhcATHxK2SLNeP5W3v1v']


In [12]:
print(len(ids))

9846


In [13]:
def get_track_features(track_id):
    # Fetch metadata and audio features
    meta = sp.track(track_id)
    features = sp.audio_features(track_id)[0]
    
    # Extract metadata
    track_info = {
        'track_id': track_id,
        'name': meta.get('name'),
        'album': meta.get('album', {}).get('name'),
        'artist': meta.get('album', {}).get('artists', [{}])[0].get('name'),
        'release_date': meta.get('album', {}).get('release_date'),
        'length': meta.get('duration_ms'),
        'popularity': meta.get('popularity')
    }
    
    # Extract audio features
    feature_keys = ['acousticness', 'danceability', 'energy', 'instrumentalness', 
                    'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature']
    
    for key in feature_keys:
        track_info[key] = features.get(key)
    
    # Convert track info to a list
    track = [
        track_info['track_id'],
        track_info['name'],
        track_info['album'],
        track_info['artist'],
        track_info['release_date'],
        track_info['length'],
        track_info['popularity'],
        track_info['danceability'],
        track_info['acousticness'],
        track_info['energy'],
        track_info['instrumentalness'],
        track_info['liveness'],
        track_info['loudness'],
        track_info['speechiness'],
        track_info['tempo'],
        track_info['time_signature']
    ]
    
    return track


In [14]:
# Function to split the list into n sublists
def split_list(input_list, n):
    k, m = divmod(len(input_list), n)
    return [input_list[i*k + min(i, m):(i+1)*k + min(i+1, m)] for i in range(n)]

# Splitting the list into 99 sublists
sublists = split_list(ids, 100)

# Checking the length of the sublists
for i, sublist in enumerate(sublists):
    print(f"Sublist {i+1} length: {len(sublist)}")

# Optional: If you need the sublists as separate variables
sublist_vars = [f'ids_{i+1}' for i in range(99)]
for var_name, sublist in zip(sublist_vars, sublists):
    globals()[var_name] = sublist

# Example: Accessing the first sublist
print(len(ids_1))
print(ids_2)
print(ids_3)



Sublist 1 length: 99
Sublist 2 length: 99
Sublist 3 length: 99
Sublist 4 length: 99
Sublist 5 length: 99
Sublist 6 length: 99
Sublist 7 length: 99
Sublist 8 length: 99
Sublist 9 length: 99
Sublist 10 length: 99
Sublist 11 length: 99
Sublist 12 length: 99
Sublist 13 length: 99
Sublist 14 length: 99
Sublist 15 length: 99
Sublist 16 length: 99
Sublist 17 length: 99
Sublist 18 length: 99
Sublist 19 length: 99
Sublist 20 length: 99
Sublist 21 length: 99
Sublist 22 length: 99
Sublist 23 length: 99
Sublist 24 length: 99
Sublist 25 length: 99
Sublist 26 length: 99
Sublist 27 length: 99
Sublist 28 length: 99
Sublist 29 length: 99
Sublist 30 length: 99
Sublist 31 length: 99
Sublist 32 length: 99
Sublist 33 length: 99
Sublist 34 length: 99
Sublist 35 length: 99
Sublist 36 length: 99
Sublist 37 length: 99
Sublist 38 length: 99
Sublist 39 length: 99
Sublist 40 length: 99
Sublist 41 length: 99
Sublist 42 length: 99
Sublist 43 length: 99
Sublist 44 length: 99
Sublist 45 length: 99
Sublist 46 length: 

In [15]:
# import time
# import random
# import pandas as pd

# def fetch_data_with_backoff(ids):
#     tracks = []
#     for track_id in ids:
#         success = False
#         max_retries = 5
#         retry_count = 0
#         backoff_time = 1  # Initial backoff time in seconds

#         while not success and retry_count < max_retries:
#             try:
#                 track = get_track_features(track_id)
#                 tracks.append(track)
#                 success = True
#             except Exception as e:
#                 print(f"Error fetching data for track {track_id}: {e}")
#                 retry_count += 1
#                 if retry_count < max_retries:
#                     sleep_time = backoff_time + random.uniform(0, 1)  # Adding jitter
#                     print(f"Retrying in {sleep_time:.2f} seconds...")
#                     time.sleep(sleep_time)
#                     backoff_time *= 2  # Exponential backoff

#     return pd.DataFrame(tracks, columns=[
#         'track_id', 'name', 'album', 'artist', 'release_date', 'length', 
#         'popularity', 'danceability', 'acousticness', 'energy', 
#         'instrumentalness', 'liveness', 'loudness', 'speechiness', 
#         'tempo', 'time_signature'
#     ])

# def process_batches():
#     all_dfs = []
    
#     for batch_num in range(1, 96, 20):  # Process 20 IDs at a time
#         ids_batch = [eval(f'ids_{i}') for i in range(batch_num, min(batch_num + 20, 96))]
        
#         for ids in ids_batch:
#             df = fetch_data_with_backoff(ids)
#             all_dfs.append(df)

#         # Sleep for 24 hours (86400 seconds) between each batch
#         print(f"Batch {batch_num} processed. Sleeping for 24 hours.")
#         time.sleep(86400)
    
#     return all_dfs

# # Start the process
# all_dfs = process_batches()
# all_dfs.to_csv('combined_track_data.csv', index=False)

# print("CSV files have been successfully combined into 'combined_track_data.csv'")


after trying multiple ways to get pass rate limiting including developing a backoff-retry strategy ive decided to do some reaserach on different ways to scrape this data. i found SpotiFile which is a scrip that allows users to more easily scrape spotify playlist,albums and artist but after continued research ive concluded that both SpotifyScraper and the above API scraping methods have depreciated due to new spotify developer guidelines (section 5). "'scraping', whether manually or by automated means, or otherwise using any automated means (including bots, scrapers, and spiders), to view, access or collect information" instead ive scoured the internet for multiple different datasets containing the information I want. Github user Logan Norman under the ALias lognorman20 provides a dataset with top spotify songs and their features.  

In [16]:


# Read the CSV file
data1 = pd.read_csv('/Users/acthach/Desktop/Summer proj/combined_track_data.csv')

# Display the first few rows of the dataframe
print(data1.head())


                 track_id                  name  \
0  4xdBrk0nFZaP54vvZj0yx7            HOT TO GO!   
1  6dOtVTDdiauQNBQEDOtlAB    BIRDS OF A FEATHER   
2  5N3hjp1WNayUPZrA8kJmJP  Please Please Please   
3  19RybK6XDbAVpcdxSbZL1o                 Apple   
4  51eSHglvG1RJXtL3qI5trr          Slow It Down   

                                     album             artist release_date  \
0  The Rise and Fall of a Midwest Princess      Chappell Roan   2023-09-22   
1                     HIT ME HARD AND SOFT      Billie Eilish   2024-05-17   
2                     Please Please Please  Sabrina Carpenter   2024-06-06   
3                                     BRAT         Charli xcx   2024-06-07   
4                 Fireworks & Rollerblades       Benson Boone   2024-04-05   

   length  popularity  danceability  acousticness  energy  instrumentalness  \
0  184841          90         0.742        0.0187   0.757            0.0000   
1  210373          99         0.747        0.2000   0.507         

In [17]:

data1['favorite'] = 0 
data1.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,4xdBrk0nFZaP54vvZj0yx7,HOT TO GO!,The Rise and Fall of a Midwest Princess,Chappell Roan,2023-09-22,184841,90,0.742,0.0187,0.757,0.0000,0.3050,-4.981,0.0421,139.982,4,0
1,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,HIT ME HARD AND SOFT,Billie Eilish,2024-05-17,210373,99,0.747,0.2000,0.507,0.0608,0.1170,-10.171,0.0358,104.978,4,0
2,5N3hjp1WNayUPZrA8kJmJP,Please Please Please,Please Please Please,Sabrina Carpenter,2024-06-06,186365,98,0.669,0.2740,0.586,0.0000,0.1040,-6.073,0.0540,107.071,4,0
3,19RybK6XDbAVpcdxSbZL1o,Apple,BRAT,Charli xcx,2024-06-07,151739,86,0.804,0.1250,0.957,0.0174,0.1140,-5.452,0.0340,126.035,4,0
4,51eSHglvG1RJXtL3qI5trr,Slow It Down,Fireworks & Rollerblades,Benson Boone,2024-04-05,161831,89,0.432,0.1740,0.583,0.0000,0.0933,-4.682,0.0687,181.489,4,0


In [18]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="XXXXXXX",
                                               client_secret="XXXXXXX",
                                               redirect_uri="http://localhost:8888/callback",
                                               scope="user-top-read"))

ranges = ['medium_term']


for sp_range in ranges:
    print("range:", sp_range)
    results1 = sp.current_user_top_tracks(time_range=sp_range, limit=50)
    for i, item in enumerate(results1['items']):
        print(i, item['name'], '//', item['artists'][0]['name'])
    print()


range: medium_term


0 Dawn // Eliminate
1 Cottage Gore // Subtronics
2 Spacetime - Virtual Riot Remix // Subtronics
3 far away - Crankdat Remix // Mark Tuan
4 Under Control (feat. Hurts) // Calvin Harris
5 Huntin’ Wabbitz // J. Cole
6 Black Out Days - Subtronics Remix // Phantogram
7 Wake Me Up // Avicii
8 how2fly // ISOxo
9 Fever // J. Cole
10 Back Where I Belong (feat. Avicii) // Otto Knows
11 Too Sweet // Hozier
12 Feel It - From The Original Series “Invincible” // d4vd
13 Not Even Love // Seven Lions
14 Carry You // Martin Garrix
15 Hymn for the Weekend - Seeb Remix // Coldplay
16 far away // Mark Tuan
17 places to be // Fred again..
18 overwhelmed - Chri$tian Gate$ remix // Royal & the Serpent
19 Alien Communication // Subtronics
20 Evil // Doov
21 SOS (feat. Aloe Blacc) // Avicii
22 In the Name of Love // Martin Garrix
23 LAST OF US (feat. Rita Ora) (Jerro Remix) // Gryffin
24 Not Like Us // Kendrick Lamar
25 Saturn // SZA
26 Happier By Now // Kai Wachi
27 I Wish I Was // The Stickmen Project
28 Nob

In [19]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results1['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Dawn,Dawn,7aXAO9hfBlyAt8VdQwjPje,Eliminate,223255,32
1,Cottage Gore,TESSERACT,4aZ9YyxKfek4k2Gfm4eeFs,Subtronics,164068,49
2,Spacetime - Virtual Riot Remix,ANTIFRACTALS,5VJuzCF7u2Q7G9hYvtd6f6,Subtronics,210206,39
3,far away - Crankdat Remix,far away (Crankdat Remix),4fj1ITLZ06PioSOZK8rhUc,Mark Tuan,203076,39
4,Under Control (feat. Hurts),Motion,4J7CKHCF3mdL4diUsmW8lq,Calvin Harris,184280,78


In [20]:
fav_tracks = []
for track in df_favourite['track_id']:
    track = get_track_features(track)
    fav_tracks.append(track)


In [21]:
df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_fav['favorite'] = 1
df_fav.head()



,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,7aXAO9hfBlyAt8VdQwjPje,Dawn,Dawn,Eliminate,2024-01-12,223255,32,0.527,0.10900,0.944,0.799000,0.4000,-0.669,0.3580,172.142,4,1
1,4aZ9YyxKfek4k2Gfm4eeFs,Cottage Gore,TESSERACT,Subtronics,2024-02-16,164068,49,0.641,0.00472,0.898,0.535000,0.2190,-0.875,0.3250,145.049,4,1
2,5VJuzCF7u2Q7G9hYvtd6f6,Spacetime - Virtual Riot Remix,ANTIFRACTALS,Subtronics,2022-12-16,210206,39,0.626,0.01690,0.895,0.000352,0.2850,-2.136,0.0785,144.970,4,1
3,4fj1ITLZ06PioSOZK8rhUc,far away - Crankdat Remix,far away (Crankdat Remix),Mark Tuan,2022-12-02,203076,39,0.601,0.13000,0.728,0.000000,0.0846,-2.944,0.0400,156.169,4,1
4,4J7CKHCF3mdL4diUsmW8lq,Under Control (feat. Hurts),Motion,Calvin Harris,2014-10-31,184280,78,0.544,0.12900,0.915,0.000914,0.1150,-3.405,0.0841,126.094,4,1


In [22]:
# Dropping columns that could lead to data leakage
df_fav = df_fav.drop(columns=['name', 'album', 'artist', 'release_date'])
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,7aXAO9hfBlyAt8VdQwjPje,223255,32,0.527,0.10900,0.944,0.799000,0.4000,-0.669,0.3580,172.142,4,1
1,4aZ9YyxKfek4k2Gfm4eeFs,164068,49,0.641,0.00472,0.898,0.535000,0.2190,-0.875,0.3250,145.049,4,1
2,5VJuzCF7u2Q7G9hYvtd6f6,210206,39,0.626,0.01690,0.895,0.000352,0.2850,-2.136,0.0785,144.970,4,1
3,4fj1ITLZ06PioSOZK8rhUc,203076,39,0.601,0.13000,0.728,0.000000,0.0846,-2.944,0.0400,156.169,4,1
4,4J7CKHCF3mdL4diUsmW8lq,184280,78,0.544,0.12900,0.915,0.000914,0.1150,-3.405,0.0841,126.094,4,1


In [23]:
data1 = data1.drop(columns=['name', 'album', 'artist', 'release_date'])
data1.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,4xdBrk0nFZaP54vvZj0yx7,184841,90,0.742,0.0187,0.757,0.0000,0.3050,-4.981,0.0421,139.982,4,0
1,6dOtVTDdiauQNBQEDOtlAB,210373,99,0.747,0.2000,0.507,0.0608,0.1170,-10.171,0.0358,104.978,4,0
2,5N3hjp1WNayUPZrA8kJmJP,186365,98,0.669,0.2740,0.586,0.0000,0.1040,-6.073,0.0540,107.071,4,0
3,19RybK6XDbAVpcdxSbZL1o,151739,86,0.804,0.1250,0.957,0.0174,0.1140,-5.452,0.0340,126.035,4,0
4,51eSHglvG1RJXtL3qI5trr,161831,89,0.432,0.1740,0.583,0.0000,0.0933,-4.682,0.0687,181.489,4,0


In [24]:
# Checking for duplicates in df_fav
df_fav['track_id'].value_counts()

track_id
7aXAO9hfBlyAt8VdQwjPje    1
2Mwmp4qVCe8CJcalsT4xm1    1
37gvCv1XpbCfG3Acjqf6ew    1
47N81NMkB488fuOwOC3Oip    1
5MLHXEDZKQLfh8ljT4JMVG    1
4S8PxReB1UiDR2F5x1lyIR    1
2qDux6mBTKsUwPIPG5Leay    1
0fu9Vvp8bOBtB1IEv7JHsB    1
4a83gGp7WNs0UzwzKPCY4C    1
1P7hMSbpJOQy58eC78qx7s    1
23tSVCziOnXVAyayQt0Bi7    1
5w9upngVRHNjdZcRC7Xxr2    1
68R0zVUeMJ2C852Ov6d2Mh    1
4aZ9YyxKfek4k2Gfm4eeFs    1
3rFHJDslBgt9rHu8p22aFN    1
2r5QrlLv0WqAzEpsoI2zzA    1
37ouI56S9L2R8ASWeyAMG7    1
17hgE2h9PhLC6RJuFm2BEQ    1
1ITpqT1kDATCtMBwOK7sYn    1
1j2iMeSWdsEP5ITCrZqbIL    1
6NRvZuFXn2ixp8YdzUvG5n    1
7u8cR8A8XVdrk24vMYjhV2    1
0bTcsfvvELwAupgm0DxhjK    1
1VviocEvd6zv4xudscXgcj    1
0qq33nVMNHelvjRGhTKPVH    1
1bjeWoagtHmUKputLVyDxQ    1
6AI3ezQ4o3HUoP6Dhudph3    1
24fSBi06Sl89i2mPpl4jvz    1
5VJuzCF7u2Q7G9hYvtd6f6    1
4fj1ITLZ06PioSOZK8rhUc    1
4J7CKHCF3mdL4diUsmW8lq    1
2cH8wHImerFokgmhjyhCTU    1
0Tcy9RRWvcnqepvmVcqv05    1
0nrRP2bk19rLc0orkWPQk2    1
2fga4JgHuq91W9HiTfo9GC    1
6XkIQqAsFjk

In [25]:
print(data1.shape)
print(df_fav.shape)

(9347, 13)
(50, 13)


In [26]:
data1.columns == df_fav.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [27]:
combined = pd.concat([data1, df_fav])
combined.shape

(9397, 13)

In [28]:
combined.favorite.value_counts()

favorite
0    9347
1      50
Name: count, dtype: int64

In [29]:
df_fav.to_csv('favorite_songs.csv', index=False)
data1.to_csv('spotify_topsongs.csv', index = False)